In [111]:
import config
import gspread
import pandas as pd

gc = gspread.service_account()
sheet = gc.open("Optic Lab Shooting schedule")
            
#worksheet_list = sh.worksheets()
#print(worksheet_list)
#print(sh.sheet1.get('A1'))


temp_worksheet = sheet.worksheet("temperaturas")
temp_dataframe = pd.DataFrame(temp_worksheet.get_all_records())
video_worksheet = sheet.worksheet("FINAL EDITS")


In [112]:
values_list = video_worksheet.row_values(1)
values_list


['USE',
 'FILE NAME',
 'FTS',
 'DURATION',
 'CHANNEL',
 'MOVE',
 'NOTES',
 'SCALE',
 'FEELING',
 'ROOM / DEVICE',
 'STAGE',
 'MOTION',
 'LIGHT',
 'COLOR',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'w']

In [113]:
video_dataframe = pd.DataFrame(video_worksheet.get_all_records())

In [114]:
video_dataframe.head(2)


,USE,FILE NAME,FTS,DURATION,CHANNEL,MOVE,NOTES,SCALE,FEELING,ROOM / DEVICE,...,COLOR,O,P,Q,R,S,T,U,V,w
0,Y,A003_04290822_C037.mov,,0:29:20,BOB,,scientists en el lab clean lab de espalda,landscape,caring,Diamond Acid,...,-,ALTA,,,,,MILD,,,
1,Y,A003_04290254_C006_B.mov,,0:30:12,ALICE,,Clean room close up puring acid shaky hand,human,caring,Diamond Acid,...,,,,,,Cine,MILD,,,


In [131]:
#video_dataframe.filter(like='Y', axis=1)
la = video_dataframe.query('USE == "Y" and CHANNEL == "BOB" and FEELING == "caring"')

la.head(13)

,USE,FILE NAME,FTS,DURATION,CHANNEL,MOVE,NOTES,SCALE,FEELING,ROOM / DEVICE,...,COLOR,O,P,Q,R,S,T,U,V,w
0,Y,A003_04290822_C037.mov,,0:29:20,BOB,,scientists en el lab clean lab de espalda,landscape,caring,Diamond Acid,...,-,ALTA,,,,,MILD,,,
3,Y,A003_04290307_C009_D.mov,,0:43:13,BOB,,clean room cleaning small jars,human,caring,Diamond Acid,...,,stabilized,,,,Cine,MILD,,,
4,Y,A003_04290304_C007_A.mov,,0:34:22,BOB,,Clean room two small jars hands move them,human,caring,Diamond Acid,...,,,,,,Cine,MILD,,,
9,Y,A003_04290740_C021_B.mov,,0:20:00,BOB,,diamond a bit out of focus,landscape,caring,Growth,...,,,,,,Cine,MILD,,,
10,Y,A003_04290729_C020.mov,,0:28:00,BOB,,diamond throu mirror,landscape,caring,Growth,...,,,,,,Cine,MILD,,,
12,Y,A003_04290728_C019.mov,,0:42:07,BOB,,diamond throu mirror,landscape,caring,Growth,...,,,,,,Cine,MILD,,,


In [121]:
la['FILE NAME'].tolist()

['A003_04290822_C037.mov',
 'A003_04290254_C006_B.mov',
 'A003_04290254_C006_A.mov',
 'A003_04290307_C009_D.mov',
 'A003_04290304_C007_A.mov',
 'A003_04290249_C005_B.mov',
 'A003_04290254_C006_C.mov',
 'A003_04290249_C005_A.mov',
 'A003_04290740_C021_A.mov',
 'A003_04290740_C021_B.mov',
 'A003_04290729_C020.mov',
 'A001_04290632_C063.mov',
 'A003_04290728_C019.mov']

In [59]:
temp_dataframe.head(9)

,CATEGORIA,Amount:,Aprox Total Time,temp difference,runtime per degree,temp_start,temp_end,Sound:,Description:
0,CARING,13,9:38:00,4,2:24:30,41,100,,Diamond Growth and Piranha Clean Room
1,LONELY,36,10:14:00,4,2:33:30,37,41,,Route and Abandoned KPN
2,STABILITY,20,8:19:00,3,2:46:20,34,37,,Clocks and Diamond Model
3,WAITING,15,8:13:00,3,2:44:20,31,34,,Clocks - Machines - Estanterías - Samples
4,PRESSURE,10,3:50:00,2,1:55:00,29,31,,Nitrogen and some clocks
5,SUSPENSE,16,9:09:00,4,2:17:15,25,29,,Optical Table and Golden Tower
6,FLIRTING,24,7:28:00,3,2:29:20,22,25,,Diamante microscopio
7,EXCITEMENT,31,13:07:00,4,3:16:45,18,22,,Laser
8,COMMITMENT,10,4:54:00,2,2:27:00,-100,18,,Cube and Optical Table


In [73]:
df1 = pd.DataFrame({
    'Value': {
        0: 45, 
        1: 38, 
        2: 35, 
        3: 16}, 
})

df = temp_dataframe.head(9).astype({"temp_start":"int","temp_end":"int"})

s1=df.temp_start.values
s2=df.temp_end.values
s=df1.Value.values[:,None]
np.dot((s>=s1)&(s<=s2),df.CATEGORIA)




0        CARING
1        LONELY
2     STABILITY
3       WAITING
4      PRESSURE
5      SUSPENSE
6      FLIRTING
7    EXCITEMENT
8    COMMITMENT
Name: CATEGORIA, dtype: object

In [82]:
def cat_from_temp(df, temp):
    tempf = pd.DataFrame({ 'Value': { 0: temp }})
    s1=df.temp_start.values
    s2=df.temp_end.values
    s=tempf.Value.values[:,None]
    return np.dot((s>=s1)&(s<=s2),df.CATEGORIA)[0]
    
cat = cat_from_temp(df, 12)


In [72]:
import numpy as np

def dataframe_to_code(df):
    data = np.array2string(df.to_numpy(), separator=', ')
    data = data.replace(" nan", " float('nan')")
    cols = df.columns.tolist()
    return f"""pd.DataFrame({data}, columns={cols})"""

temps = 'test.txt'

with open(temps, "w") as f:
    f.write(dataframe_to_code(df))

with open (temps, "r") as f:
    imp = eval(f.read())
imp

,CATEGORIA,Amount:,Aprox Total Time,temp difference,runtime per degree,temp_start,temp_end,Sound:,Description:
0,CARING,13,9:38:00,4,2:24:30,41,100,,Diamond Growth and Piranha Clean Room
1,LONELY,36,10:14:00,4,2:33:30,37,41,,Route and Abandoned KPN
2,STABILITY,20,8:19:00,3,2:46:20,34,37,,Clocks and Diamond Model
3,WAITING,15,8:13:00,3,2:44:20,31,34,,Clocks - Machines - Estanterías - Samples
4,PRESSURE,10,3:50:00,2,1:55:00,29,31,,Nitrogen and some clocks
5,SUSPENSE,16,9:09:00,4,2:17:15,25,29,,Optical Table and Golden Tower
6,FLIRTING,24,7:28:00,3,2:29:20,22,25,,Diamante microscopio
7,EXCITEMENT,31,13:07:00,4,3:16:45,18,22,,Laser
8,COMMITMENT,10,4:54:00,2,2:27:00,-100,18,,Cube and Optical Table
